In [41]:
import os, json
import random
import math

In [42]:
random.seed(42)
pk = 3

In [43]:
original_data_path = '../Datasets/pandasNumpyEval/offical_numpy.jsonl'
result_data_path = '../results/pandasNumpyEval_numpy_heuristic_results.json'
combined_data_path = '../Combined_Datasets/pandasNumpyEval_numpy_combined.json'

In [44]:
original_data = []
with open(original_data_path) as f:
    for line in f:
        original_data.append(json.loads(line))

len(original_data)

101

In [45]:
result_data = []
with open(result_data_path) as f:
    data = json.load(f)
    extra = 0
    for i in range(len(data)):
        if extra > 0:
            extra -= 1
            continue
        new_data = data[i]
        new_comment= data[i]['nl']['comment']
        new_heuristics = data[i]['Heuristic']
        for j in range(i+1, len(data)):
            if data[i]['nl']['id'] == data[j]['nl']['id'] :
                new_comment = new_comment+"\n"+ data[i]['nl']['comment']
                new_heuristics = new_heuristics + data[j]['Heuristic']
                extra += 1
            else:
                break
        
        new_heuristics = list(set(new_heuristics))
        new_data['nl']['comment'] = new_comment
        new_data['Heuristic'] = new_heuristics
        result_data.append(new_data)

len(result_data)
assert len(result_data) == len(original_data)

In [46]:
def get_statistical_significance_samples_ids(population_size, confidence_interval, error_margin):
    """
    Get the sample ids for the statistical significance test.
    """
    z_score = 0
    if confidence_interval == 0.95:
        z_score = 1.96
    elif confidence_interval == 0.99:
        z_score = 2.58
    no_top = (z_score ** 2) * 0.25
    no_bottom = error_margin ** 2
    no = no_top / no_bottom
    sample_size = math.ceil(no / (1 + no / population_size))

    # Generate random sample IDs
    sample_ids = random.sample(range(0, population_size), int(sample_size))
    
    return sample_ids

    

In [47]:
sample_ids = get_statistical_significance_samples_ids(len(original_data), 0.99, 0.05)
print(len(sample_ids))

88


In [48]:
def fix_heuristic_list(heuristic_list):
    result = []
    for heuristic in heuristic_list:
        result.append(int(heuristic.replace('H', '')))
    return result

In [49]:
combined_data = []
for i in range(len(original_data)):
    if i not in sample_ids:
        continue
    original = original_data[i]
    result = result_data[i]
    original['nl'] = result['nl']['comment'] 
    combined_data.append({
        "model": "datasets_study.problem",
        "pk": pk,
        "fields": {
            "source_dataset": "CoderEval4Python",
            "prompt_id": result['nl']['id'],
            "content": original,
            "language": "py",
            "problems": fix_heuristic_list(result['Heuristic']),
        }

    })
    pk += 1



In [50]:
with open(combined_data_path, 'w') as f:
    json.dump(combined_data, f, indent=4)